<a href="https://colab.research.google.com/github/lolanzo/lolanzo/blob/main/PRiSM_SampleRNN_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRiSM SampleRNN
![alt text](https://drive.google.com/uc?export=view&id=18hZ4FCNCgxthTFAxFV-iCvYYNJBS3ZS5)

### Neural Sound Synthesis with TensorFlow 2

Dr Christopher Melen, [RNCM PRiSM](https://www.rncm.ac.uk/research/research-centres-rncm/prism/)

*Last update: October 9th, 2020*









For more information on neural synthesis, SampleRNN and Machine Learning in general see the PRiSM blog article [A Short History of Neural Synthesis](https://www.rncm.ac.uk/research/research-centres-rncm/prism/prism-blog/a-short-history-of-neural-synthesis/).

If you are new to Google Colaboratory ('Colab' for short) please visit the [official overview](https://colab.research.google.com/notebooks/basic_features_overview.ipynb), and also be sure to consult the [FAQ](https://research.google.com/colaboratory/faq.html).

This notebook does not assume advanced programming knowledge, but some familiarity with basic Unix shell commands and/or running Python scripts would be beneficial. In Colab shell commands are prefixed with `!` or `%`, for example `%cd some_folder` to change directory. This comes from the special syntax used by [Jupyter Notebooks](https://jupyter.org/), on which Google Colab is based.

In order to execute code cells in this notebook you will require a Google account, and be signed in. For best results we recommend running it in Google's [Chrome](https://www.google.com/intl/en/chrome/) web browser.

**Please note** that due to the [resource limits](https://research.google.com/colaboratory/faq.html#resource-limits) of the Colab environment this notebook is provided only as a means of testing out PRiSM SampleRNN, and discovering what it has to offer. To use the code in a 'production' environment you will require access to a machine equipped with an NVIDIA GPU, and at least 8GB of RAM, or machine learning resources provided by Cloud platforms such as [Google Cloud](https://cloud.google.com/gpu) or [AWS](https://docs.aws.amazon.com/dlami/latest/devguide/gpu.html).

## Getting Started

Copy the notebook to your Google Drive with **File -> Save a copy in Drive** , so that any changes you make will be saved.

In order to run the code cells in the notebook you will first need to make the GitHub repository available to the notebook by saving it to your Google Drive, which can be mounted to the notebook. To mount your Drive click the **Files** icon in the sidebar to the left, then in the row of buttons that appears at the top of the sidebar click **Mount Drive** on the right. For this notebook we are assuming the code will be accessible from a folder called `prism-samplernn` at the top level of your Drive.

To ensure that you are at the top level of your Drive execute:



In [1]:
%cd drive/My\ Drive

[Errno 2] No such file or directory: 'drive/My Drive'
/content


The recommended way to get the code is by cloning it using [Git](https://en.wikipedia.org/wiki/Git). To do that execute the following:

In [ ]:
!git clone https://github.com/rncm-prism/prism-samplernn.git

The main benefit of using Git (or indeed any [version control system](https://www.git-tower.com/learn/git/ebook/en/command-line/basics/why-use-version-control/)) is that you can easily keep up to date with new versions of the code. Colab comes with Git conveniently pre-installed. The repository can be updated at any time using the `git pull` command from inside the folder:



In [ ]:
%cd /content/drive/My\ Drive/prism-samplernn/
!git pull

Alernatively you can download an archive of the code, although note that the resulting folder will *not* be a Git archive, and cannot be kept up to date using Git. The following commands will download a zip archive of the code using `wget`, and then unpack it (**don't** execute this if you have already obtained the code using Git!):

In [ ]:
!wget https://github.com/rncm-prism/prism-samplernn/archive/master.zip
!unzip master.zip
!mv prism-samplernn-master prism-samplernn

Although PRiSM SampleRNN requires [TensorFlow 2](https://www.tensorflow.org/) we do not have to install that as Colab aleady provides it for us. To ensure we are running this version execute:

In [ ]:
%tensorflow_version 2.x

Then install the remaining dependencies:

In [ ]:
!pip install librosa
!pip install natsort
!pip install pydub

We also need to install the Python soundfile library, which Librosa should automatically install, but unfortunately doesn't on Colab:

In [ ]:
!pip install soundfile

## GPU

For optimal performance it is necessary to run the code using a GPU. A range of NVIDIA GPUs are currently provided by Colab, including the K80 and the T4 models. Although it is not possible to choose the type of GPU connected to a notebook, we recommend the T4 due to its greater speed and memory. To verify the currently available GPU run the Nvidia System Management Interface with:

In [ ]:
!nvidia-smi

Wed Jul  1 12:17:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Creating a Dataset

To train the model we need some input data, which in this case should be a single **mono** wav file sliced into equal length chunks, each chunk being about 5-10 seconds in duration. Given the obvious limitations of the Colab environment we recommend this source wav be no more than about 20-30 minutes in duration, with a sample rate of 11025.

To create such a dataset:

1.   Upload the source wav file to the Google Drive directory containing the code.
2.   Create a new directory to hold the wav chunks.
3.   Run the supplied `chunk_audio.py` script.

Before we can run this or any of the other scripts, however, we need to ensure we are in the directory containing the code:







In [ ]:
%cd /content/drive/My\ Drive/prism-samplernn/

Now create a folder in which to store the chunks... here we're simply calling the folder `chunks`, but you can use whatever name you like:

In [ ]:
%mkdir chunks

To create the chunks execute the command below. Note the `chunk_length` parameter, which sets the size of the output chunks in milliseconds. We're passing 8000 as the value below, so each chunk will be 8 seconds in duration.

In [ ]:
!python chunk_audio.py --input_file test.wav --output_dir ./chunks/ --chunk_length 8000

## TensorBoard

TensorFlow provides a very useful graphical tool called [TensorBoard](https://www.tensorflow.org/tensorboard) for monitoring important training metrics such as loss and accuracy. To enable it execute the following:

In [ ]:
%load_ext tensorboard

Then load TensorBoard into the notebook with the following command:

In [ ]:
%tensorboard --logdir logdir/default/train

The TensorBoard UI is a 'live' interface which will update automatically every thirty seconds (the update interval can be configured from inside TensorBoard).

## Model Checkpoints

While the training script is running the current version of the model is periodically saved to disk under the session log directory. These 'checkpoint' files store snapshots of the state of the model, persisting it beyond the lifetime of the training session. They are also used to automatically pick up an interrupted training session from the last saved version, when the `resume` parameter is set to `True` (the default).

Checkpoints are by default saved at the end of every epoch. This interval can be increased, however, using the `checkpoint_every` parameter.

Since checkpoints can be quite large, often growing to a Gigabyte or more, they are automatically deleted, with only a rolling 'window' of checkpoints prior to the the current checkpoint being retained. The size of this window is configurable through the `max_checkpoints` parameter to the training script, which by default is set to 5.

Deleted checkpoints are sent to the Google Drive bin, but **please be aware this does not permanently delete them from Drive**. Files accumulated in the bin will still contribute to Drive storage, and could eventually exhaust it. These files must therefore be removed permanently, which can be done in either of the following ways:

1.   Manual deletion, by periodically checking the Google Drive bin and clearing it out. To do this click the bin icon to view its contents and select **Bin -> Empty bin**.
2.   Automatically clear the bin at set intervals using a Google Drive API script. Detailed instructions on how to set up such a script can be found [here](https://gist.github.com/relativeflux/032d4087245cf8aa5b21eccfbf836435).

A single model checkpoint is stored on disk as multiple files, conforming to the general naming pattern `model.ckpt-n.*`, where `n` is the epoch number. Note, however, that when referencing checkpoints they are treated as single files, using the `model.ckpt-n` pattern.



## Running the Training Script

Once we have successfully generated our dataset of audio chunks we can move on to running the training script, `train.py`. The following table lists the hyperparameters that may be passed to the script at the command line:

| Parameter Name             | Description           | Default Value  | Required?   |
| ---------------------------|-----------------------|----------------| -----------|
| `id`                     | Id for the training session.          | `default`           | No        |
| `data_dir`               | Path to the directory containing the training data.           | `None`           | Yes        |
| `verbose`                | Set training output verbosity. If `False` training step output is overwritten, if `True` (the default) it is written to a new line.           | `None`           | No        |
| `logdir_root`            | Location in which to store training log files and checkpoints. All such files are placed in a subdirectory with the id of the training session.           | `./logdir`           | No      |
| `output_dir`             | Path to the directory for audio generated during training.           | `./generated`           | No      |
| `config_file`            | File containing the configuration parameters for the training model. Note that this file must contain valid JSON, and should have a name that conforms to the `*.config.json` pattern. | `./default.config.json`         | No        |
| `num_epochs`             | Number of epochs to run the training. | 100           | No        |
| `batch_size`             | Size of the mini-batch. It is recommended that the batch size divide the length of the training corpus without remainder, otherwise the dataset will be truncated to the nearest multiple of the batch size. | 64         | No        |
| `optimizer`              | TensorFlow optimizer to use for training. (`adam`, `sgd` or `rmsprop`) | `adam`        | No        |
| `learning_rate`          | Learning rate of the training optimizer.   | 0.001         | No        |
| `reduce_learning_rate_after`          | Exponentially reduce learning rate after this many epochs.   | `None`         | No        |
| `momentum`               | Momentum of the training optimizer (applies to `sgd` and `rmsprop` only).   | 0.9      | No        |
| `checkpoint_every`       | Interval (in epochs) at which to generate a checkpoint file. Defaults to 1, for every epoch.   | 1      | No        |
| `checkpoint_policy`      | Policy for saving checkpoints - `Always` to save at the epoch interval determined by the value of `checkpoint_every`, or `Best` to save only when the loss and accuracy have improved since the last save.   | `All`      | No        |
| `max_checkpoints`        | Maximum number of checkpoints to keep on disk during training. Defaults to 5. Pass `None` to keep all checkpoints.   | 5      | No        |
| `resume`                 | Whether to resume training, either from the last available checkpoint or from one supplied using the `resume_from` parameter.   | `True`      | No        |
| `resume_from`            | Checkpoint from which to resume training. Ignored when `resume` is `False`.   | `None`      | No        |
| `early_stopping_patience`| Number of epochs with no improvement after which training will be stopped.   | 3      | No        |
| `generate`               | Whether to generate audio output during training. Generation is aligned with checkpoints, meaning that audio is only generated after a new checkpoint has been created.   | `True`      | No        |
| `max_generate_per_epoch` | Maximum number of output files to generate at the end of each epoch.   | 1      | No        |
| `sample_rate`            | Sample rate of the generated audio. | 22050         | No        |
| `output_file_dur`        | Duration of generated audio files (in seconds). | 3         | No        |
| `temperature`            | Sampling temperature for generated audio. | 0.75         | No        |
| `seed`                   | Path to audio for seeding when generating audio. | `None`         | No        |
| `seed_offset`            | Starting offset of the seed audio. | 0         | No        |
| `num_val_batches`               | Number of batches to reserve for validation. | 1         | No        |





Model parameters are specified through a JSON configuration file, which may be passed to the training script through the `--config_file` parameter (defaults to the supplied `default.config.json`). The following table lists the available model parameters (note that all parameters are optional and have defaults):

| Parameter Name           | Description           | Default Value  |
| -------------------------|-----------------------|----------------|
| `seq_len`                | RNN sequence length. Note that the value must be evenly-divisible by the top tier frame size.        | 1024           |
| `frame_sizes`            | Frame sizes (in samples) of the two upper tiers in the architecture, in ascending order. Note that the frame size of the upper tier must be an even multiple of that of the lower tier.  | [16,64]            |
| `dim`                    | RNN hidden layer dimensionality          | 1024         | 
| `rnn_type`         | RNN type to use, either `gru` or `lstm`           | `gru`           |
| `num_rnn_layers`         | Depth of the RNN in each of the two upper tiers           | 4          |
| `q_type`                 | Quantization type (`mu-law` or `linear`)          | `mu-law`          |
| `q_levels`               | Number of quantization channels (note that if `q_type` is `mu-law` this parameter is ignored, as mu-law quantization requires 256 channels)     | 256           |
| `emb_size`               | Size of the embedding layer in the bottom tier (sample-level MLP)         | 256          |
| `skip_conn`               | Whether to add skip connections to the model's RNN layers         | `False`          |

To run the script execute:

In [ ]:
!python train.py \
  --data_dir ./chunks \
  --num_epochs 20 \
  --batch_size 64 \
  --max_checkpoints 2 \
  --checkpoint_every 10 \
  --output_file_dur 5 \
  --sample_rate 11025

A training session that has been halted, perhaps by `Ctrl-C`, may be resumed from a previously saved checkpoint. The weights saved to the checkpoint will be loaded into a fresh model, resuming at the last epoch + 1. To enable this set `--resume` to `True`, and optionally the path to a checkpoint through the `--resume_from` parameter (ignored when `--resume` is `False`). If no such checkpoint is supplied the program will search through any previous training run directories for the latest checkpoint. If no checkpoint is found training will begin again from scratch, at epoch 1.

## Generating Audio

When we are done training we can use the `generate.py` script to generate new audio based on a saved model. The command line parameters to the script are as follows:

| Parameter Name             | Description           | Default Value  | Required?   |
| ---------------------------|-----------------------|----------------| -----------|
| `output_path`              | Path to the generated .wav file.          | `None`           | Yes        |
| `checkpoint_path`          | Path to a saved checkpoint for the model.           | `None`           | Yes        |
| `config_file`              | Path to the JSON config for the model.          | `None`           | Yes        |
| `dur`                      | Duration of generated audio.           | 3           | No       |
| `num_seqs`                 | Number of audio sequences to generate.          | 1           | No        |
| `sample_rate`              | Sample rate of the generated audio.          | 44100           | No        |
| `temperature`              | Sampling temperature for generated audio. Multiple values may be passed, to match the number of sequences to be generated.  | 0.75         | No        |
| `seed`                     | Path to audio for seeding when generating audio. | `None`         | No        |
| `seed_offset`              | Starting offset of the seed audio. | 0         | No        |

In [ ]:
!python generate.py \
  --output_path ./generated/default/test.wav \
  --checkpoint_path ./logdir/default/20.12.2020_06.15.46/model.ckpt-14 \
  --dur 10 \
  --sample_rate 11025 \
  --temperature 0.9 \
  --config_file ./default.config.json